# IE 306.02 Assignment 1 

## Call Center Simulation

Imports of random, math, numpy and simpy:

In [ ]:
import simpy
import random
import numpy as np
import math

Initializations of the constant parameters:

In [ ]:
# CONSTANTS
PATIENCE_MEAN = 60
INTER_ARRIVAL_TIME_MEAN = 14.3
FRONT_DESK_OPERATOR_SERVICE_TIME_MEAN = 7.2
FRONT_DESK_OPERATOR_SERVICE_TIME_STD = 2.7
FRONT_DESK_OPERATOR_MU = math.log(FRONT_DESK_OPERATOR_SERVICE_TIME_MEAN) - 0.5 * math.log(
    (FRONT_DESK_OPERATOR_SERVICE_TIME_STD / FRONT_DESK_OPERATOR_SERVICE_TIME_MEAN) ** 2 + 1)
FRONT_DESK_OPERATOR_SIGMA = (math.log(
    (FRONT_DESK_OPERATOR_SERVICE_TIME_STD / FRONT_DESK_OPERATOR_SERVICE_TIME_MEAN) ** 2 + 1)) ** 0.5
EXPERT_OPERATOR_SERVICE_TIME_MEAN = 10.2
EXPERT_OPERATOR_BREAK_RATE = 60
EXPERT_OPERATOR_BREAK_TIME = 3
EXPERT_OPERATOR_POISSON_THRESHOLD = math.exp(-1 * EXPERT_OPERATOR_BREAK_RATE)
SHIFT_DURATION = 480
CUSTOMER_COUNT = 125

Declaration of parameters that are going to be used during the whole process:

In [ ]:
# STAT HOLDERS
front_desk_operator_busy_time = 0
expert_operator_busy_time = 0
front_desk_operator_waiting_times = [0] * CUSTOMER_COUNT
expert_operator_waiting_times = [0] * CUSTOMER_COUNT
total_system_times = [0] * CUSTOMER_COUNT
total_shift_time = 0

#### The function that generates poisson distribution

In [ ]:
def poisson():
    n, P = 0, 1
    while True:
        P *= random.random()
        if P < EXPERT_OPERATOR_POISSON_THRESHOLD:
            return n
        n += 1

### Customer Class Definition

In [ ]:
class Customer:
    customers_served_or_reneged = CUSTOMER_COUNT

    def __init__(self, id, environment, front_desk_operator, expert_operator):
        self.id = id
        self.name = f'Customer {self.id}'
        self.environment = environment
        self.front_desk_operator = front_desk_operator
        self.expert_operator = expert_operator
        self.action = self.environment.process(self.call())

    def call(self):
        global front_desk_operator_busy_time, expert_operator_busy_time, \
            front_desk_operator_waiting_times, expert_operator_waiting_times, total_system_times, total

        print(f'{self.name} initiated call at {self.environment.now}') #prints the customer number and calling time
        total_system_time_start = self.environment.now #Total system time has now started
        front_desk_operator_wait_start = self.environment.now #Front desk operator waiting time has now started
        with self.front_desk_operator.request() as request:
            yield request

            front_desk_operator_wait_end = self.environment.now #Waiting front desk operator finished now
            front_desk_operator_waiting_times[self.id] = front_desk_operator_wait_end - front_desk_operator_wait_start #Calculate and assign the value of waiting time

            front_desk_operator_busy_time_start = self.environment.now #Now started to get service from front desk
            print(f'{self.name} started talking to the front desk operator at {self.environment.now}') #prints the customer name and calling time
            yield self.environment.timeout(random.lognormvariate(FRONT_DESK_OPERATOR_MU, FRONT_DESK_OPERATOR_SIGMA))

            front_desk_operator_busy_time_end = self.environment.now #Now finished getting service from front desk
            front_desk_operator_busy_time += front_desk_operator_busy_time_end - front_desk_operator_busy_time_start #Calculate and add the value of service time to front desk operator's total service time
            print(f'{self.name} finished talking to the front desk operator at {self.environment.now}') #prints the customer name and calling time

        expert_operator_wait_start = self.environment.now #Expert operator waiting time has now started
        with self.expert_operator.request() as request:
            patience = random.expovariate(1 / PATIENCE_MEAN) #Renege time of a customer is exponentially distributed
            results = yield request | self.environment.timeout(patience) #Results is either equal to expert operator's request or renege time

            expert_operator_wait_end = self.environment.now #Waiting expert operator finished now
            expert_operator_waiting_times[self.id] = expert_operator_wait_end - expert_operator_wait_start #Calculate and assign the value of waiting time

            if request not in results: #In the case of the customer reneged
                print(f'{self.name} reneged at {self.environment.now} after waiting '
                      f'for {patience} minutes on the expert operator\'s queue')
                total_system_time_end = self.environment.now 
                total_system_times[self.id] = total_system_time_end - total_system_time_start #Calculate and assign the value of total system time
                Customer.customers_served_or_reneged -= 1 #Customer quit the system, so decrease the number of customers
                self.environment.exit()

            expert_operator_busy_time_start = self.environment.now #Now started to get service from expert operator
            print(f'{self.name} started talking to the expert operator at {self.environment.now}') #prints the customer name and expert's service start time
            yield self.environment.timeout(random.expovariate(1 / EXPERT_OPERATOR_SERVICE_TIME_MEAN)) 

            expert_operator_busy_time_end = self.environment.now #Now finished getting service from expert operator
            expert_operator_busy_time += expert_operator_busy_time_end - expert_operator_busy_time_start #Calculate and add the value of service time to expert operator's total service time
            print(f'{self.name} finished talking to the expert operator at {self.environment.now}') #prints the customer name and expert's service finish time

        total_system_time_end = self.environment.now
        total_system_times[self.id] = total_system_time_end - total_system_time_start #Calculate and assign total system time of current customer
        Customer.customers_served_or_reneged -= 1 #Customer quit the system, so decrease the number of customers

#### Customer Generator Function

In [ ]:
def generate_customers(environment, front_desk_operator, expert_operator):
    for i in range(CUSTOMER_COUNT):
        Customer(i, environment, front_desk_operator, expert_operator)
        yield environment.timeout(random.expovariate(1 / INTER_ARRIVAL_TIME_MEAN)) #Add the exponentially distributed interarrival times between the customers

#### Break Generator Function

In [ ]:
def generate_expert_breaks(environment, expert_operator):
    while True:
        next_break_time = poisson() #Next break time is calculated by Poisson Distribution
        try:
            yield environment.timeout(next_break_time) #Calling a timeout
        except simpy.Interrupt:
            environment.exit() #If unsuccessful, then exit

        with expert_operator.request() as request:
            print(f'Expert operator requested break at {environment.now} after {next_break_time} minutes')
            try:
                yield request #Expert operator requests a timeout
            except simpy.Interrupt:
                environment.exit() #If unsuccessful, then exit

            print(f'Expert operator started break at {environment.now}')
            try:
                yield environment.timeout(EXPERT_OPERATOR_BREAK_TIME) #Process the timeout of expert operator
            except simpy.Interrupt:
                environment.exit() #If unsuccessful, then exit

            print(f'Expert operator finished break at {environment.now}')

#### Shift Generator Function

In [ ]:
def generate_shifts(environment, expert_break_process):
    shift_count = 1
    while Customer.customers_served_or_reneged != 0:
        print(f'Shift {shift_count} started at {environment.now}')
        yield environment.timeout(SHIFT_DURATION) #Process the shift duration time
        print(f'Shift {shift_count} ended at {environment.now}')
        shift_count += 1 #Update number of shifts
    expert_break_process.interrupt()

    global total_shift_time
    total_shift_time = SHIFT_DURATION * (shift_count - 1)

#### Running of the environment

In [ ]:
environment = simpy.Environment()
front_desk_operator = simpy.Resource(environment, capacity=1) #There is only one front desk operator
expert_operator = simpy.Resource(environment, capacity=1) #There is only one expert operator
customer_process = environment.process(generate_customers(environment, front_desk_operator, expert_operator))
expert_break_process = environment.process(generate_expert_breaks(environment, expert_operator))
shift_process = environment.process(generate_shifts(environment, expert_break_process))
environment.run()

#### Calculation of Statistics

In [ ]:
# STATS
total_waiting_times = [front_desk_operator_waiting_times[i] + expert_operator_waiting_times[i] for i in
                       range(CUSTOMER_COUNT)]
front_desk_operator_utilization = front_desk_operator_busy_time / total_shift_time
expert_operator_utilization = expert_operator_busy_time / total_shift_time
avg_total_waiting_time = (sum(total_waiting_times)) / CUSTOMER_COUNT
max_waiting_to_system_time_ratio = max([total_waiting_times[i] / total_system_times[i] for i in range(CUSTOMER_COUNT)])
avg_expert_operator_queue_length = (sum(expert_operator_waiting_times) / CUSTOMER_COUNT) * (
            CUSTOMER_COUNT / total_shift_time)

#### Printing the Results

In [ ]:
print('***SIMULATION STATS***')
print(f'Utilization of the front-desk operator: {front_desk_operator_utilization}')
print(f'Utilization of the expert operator: {expert_operator_utilization}')
print(f'Average Total Waiting Time: {avg_total_waiting_time}')
print(f'Maximum Total Waiting Time to Total System Time Ratio: {max_waiting_to_system_time_ratio}')
print(f'Average number of people waiting to be served by the expert operator: {avg_expert_operator_queue_length}')